In [ ]:
# -*- coding: utf-8 -*-

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Py2GvZc1GwqNbrR4fuQrHFy6zBEdF1xF


import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,BatchNormalization
from tensorflow.keras.optimizers import Adam
import os
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split
import random

(x_train,y_train),(x_test,y_test)=cifar10.load_data()
x_train=x_train.astype(np.float32)/255.0
y_train=tf.keras.utils.to_categorical(y_train,10)

x_test=x_test.astype(np.float32)/255.0
y_test=tf.keras.utils.to_categorical(y_test,10)

x_val, _, y_val,_ = train_test_split(x_test, y_test, test_size=0.6, random_state=1)

input_shape = x_train.shape[1:]

g_epoch = 70
g_batch = 64

def reset_random_seeds():
   os.environ['PYTHONHASHSEED']=str(1)
   tf.random.set_seed(1)
   np.random.seed(1)
   random.seed(1)
   os.environ['TF_DETERMINISTIC_OPS'] = '1'

reset_random_seeds() 
   
print("reduced train/val size:", len(x_train), len(x_val), "input shape:", input_shape)

# to make this notebook's output stable across runs

tf.__version__

from tensorflow.keras.layers import MaxPooling2D, Dropout, Conv2D

cnn=Sequential()
cnn.add(Conv2D(64,(3,3),activation='relu',input_shape=(32,32,3)))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(128,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(Conv2D(256,(3,3),activation='relu', padding='same'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Flatten())
cnn.add(Dense(1000,activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(10,activation='softmax'))

cnn.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy'])
cnn.summary()

hist=cnn.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

g_org_res=cnn.evaluate(x_test,y_test,verbose=0)
print("Baseline 정확률은",g_org_res[1]*100)

no_class = 10


# for transfer learning only
from tensorflow.keras.applications.resnet50 import ResNet50

os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)

# for transfer learning only
transfermodel = ResNet50(weights='imagenet',include_top=False,
                    input_shape=input_shape)
#base_model.trainable=False     # it's up to you

# your model architecture
model=Sequential()
# 전처리 레이어 추가/변경 가능
model.add(transfermodel)    # for transfer learning only
model.add(Flatten())        # for transfer learning only
model.add(Dense(1000,activation='relu')) # <<-- 변경가능
model.add(Dropout(0.25))
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(100,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(no_class, activation='softmax')) # <<-- activation은 변경가능

model.compile(loss='categorical_crossentropy',optimizer=Adam(0.00002),metrics=['accuracy']) # <<-- 변경가능
model.summary()

hist=model.fit(x_train, y_train, batch_size=g_batch, epochs=g_epoch,
             validation_data=(x_val,y_val), verbose=1)

yours=model.evaluate(x_test,y_test,verbose=0)
print("Baseline vs yours: ",g_org_res[1]*100, yours[1]*100)


print('TF {0} under GPU {1}'.format(
    tf.__version__, tf.test.is_gpu_available()))
import sys
print("python version", sys.version)

reduced train/val size: 50000 4000 input shape: (32, 32, 3)
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_24 (Conv2D)          (None, 30, 30, 64)        1792      
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 15, 15, 64)       0         
 g2D)                                                            
                                                                 
 dropout_23 (Dropout)        (None, 15, 15, 64)        0         
                                                                 
 conv2d_25 (Conv2D)          (None, 15, 15, 128)       73856     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 7, 7, 128)        0         
 g2D)                                                            
                                                           

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Baseline vs yours:  75.30999779701233 80.84999918937683
TF 2.12.0 under GPU True
python version 3.9.16 (main, Dec  7 2022, 01:11:51) 
[GCC 9.4.0]
